# Import Required Packages

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [4]:
#training controls
batch_size = 32
epochs = 20
training_size = 0.8
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [5]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [6]:
# opening the dataset
dataset = csv.reader(open("MovieGenre3.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [7]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("Poster.zip", 'r')
files.extractall()
files.close()

In [8]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('Poster/*.jpg')  

In [9]:
len(flist)

1354

In [10]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [11]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre3.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,160905,http://www.imdb.com/title/tt160905,Spooky House (2002),5.4,Comedy|Family,https://images-na.ssl-images-amazon.com/images...
1,427531,http://www.imdb.com/title/tt427531,Mezzo Forte,6.9,Animation|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...
2,4175088,http://www.imdb.com/title/tt4175088,Radical Grace (2015),8.3,Documentary|News,https://images-na.ssl-images-amazon.com/images...
3,1686053,http://www.imdb.com/title/tt1686053,Apnoia (2010),5.6,Drama|Thriller,https://images-na.ssl-images-amazon.com/images...
4,22921,http://www.imdb.com/title/tt22921,Broadway to Cheyenne (1932),4.8,Western,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
1349,821638,http://www.imdb.com/title/tt821638,Bury My Heart at Wounded Knee (2007),7.2,Drama|History|Western,https://images-na.ssl-images-amazon.com/images...
1350,87835,http://www.imdb.com/title/tt87835,"Oh, God! You Devil (1984)",5.3,Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...
1351,103710,http://www.imdb.com/title/tt103710,AprÌ¬s l'amour (1992),6.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...
1352,108265,http://www.imdb.com/title/tt108265,Swing Kids (1993),6.8,Drama|Music,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [12]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [13]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
87913,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2788556,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4767340,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
363473,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
393775,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018920,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
244244,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1897945,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
377569,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,160905,http://www.imdb.com/title/tt160905,Spooky House (2002),5.4,Comedy|Family,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,427531,http://www.imdb.com/title/tt427531,Mezzo Forte,6.9,Animation|Comedy|Crime,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4175088,http://www.imdb.com/title/tt4175088,Radical Grace (2015),8.3,Documentary|News,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1686053,http://www.imdb.com/title/tt1686053,Apnoia (2010),5.6,Drama|Thriller,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,22921,http://www.imdb.com/title/tt22921,Broadway to Cheyenne (1932),4.8,Western,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1349,821638,http://www.imdb.com/title/tt821638,Bury My Heart at Wounded Knee (2007),7.2,Drama|History|Western,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
1350,87835,http://www.imdb.com/title/tt87835,"Oh, God! You Devil (1984)",5.3,Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1351,103710,http://www.imdb.com/title/tt103710,AprÌ¬s l'amour (1992),6.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1352,108265,http://www.imdb.com/title/tt108265,Swing Kids (1993),6.8,Drama|Music,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."


In [15]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [16]:
#setting the length of training data
length=int(len(flist)*training_size)
length

1083

In [17]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [18]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [19]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [20]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1083, 3, 268, 182)
1083 train samples
271 test samples


In [21]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

# Model 1: Resnet50

In [21]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        #x = self.sigmoid(x)
        return x

In [22]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
from statistics import mean

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1 or preds[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1 or preds[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/1083 (0%)]	Loss: 0.933625	Accuracy: 8.97%
Train Epoch: 0 [32/1083 (3%)]	Loss: 0.912466	Accuracy: 12.09%
Train Epoch: 0 [64/1083 (6%)]	Loss: 0.898731	Accuracy: 13.60%
Train Epoch: 0 [96/1083 (9%)]	Loss: 0.895101	Accuracy: 12.10%
Train Epoch: 0 [128/1083 (12%)]	Loss: 0.875491	Accuracy: 8.42%
Train Epoch: 0 [160/1083 (15%)]	Loss: 0.869574	Accuracy: 8.73%
Train Epoch: 0 [192/1083 (18%)]	Loss: 0.855055	Accuracy: 5.68%
Train Epoch: 0 [224/1083 (21%)]	Loss: 0.844683	Accuracy: 3.12%
Train Epoch: 0 [256/1083 (24%)]	Loss: 0.834526	Accuracy: 1.56%
Train Epoch: 0 [288/1083 (26%)]	Loss: 0.823277	Accuracy: 3.12%
Train Epoch: 0 [320/1083 (29%)]	Loss: 0.811384	Accuracy: 4.69%
Train Epoch: 0 [352/1083 (32%)]	Loss: 0.803142	Accuracy: 2.08%
Train Epoch: 0 [384/1083 (35%)]	Loss: 0.792774	Accuracy: 2.60%
Train Epoch: 0 [416/1083 (38%)]	Loss: 0.784782	Accuracy: 0.00%
Train Epoch: 0 [448/1083 (41%)]	Loss: 0.781660	Accuracy: 0.00%
Train Epoch: 0 [480/1083 (44%)]	Loss: 0.772044	Accuracy: 3.12

<ipython-input-23-0181edc6d50c>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.7088 
Accuracy: 3.33%
Train Epoch: 1 [0/1083 (0%)]	Loss: 0.709352	Accuracy: 1.56%
Train Epoch: 1 [32/1083 (3%)]	Loss: 0.707406	Accuracy: 7.29%
Train Epoch: 1 [64/1083 (6%)]	Loss: 0.706660	Accuracy: 7.29%
Train Epoch: 1 [96/1083 (9%)]	Loss: 0.706116	Accuracy: 5.73%
Train Epoch: 1 [128/1083 (12%)]	Loss: 0.707838	Accuracy: 2.60%
Train Epoch: 1 [160/1083 (15%)]	Loss: 0.702606	Accuracy: 4.69%
Train Epoch: 1 [192/1083 (18%)]	Loss: 0.704096	Accuracy: 7.29%
Train Epoch: 1 [224/1083 (21%)]	Loss: 0.702847	Accuracy: 2.60%
Train Epoch: 1 [256/1083 (24%)]	Loss: 0.700894	Accuracy: 8.85%
Train Epoch: 1 [288/1083 (26%)]	Loss: 0.700301	Accuracy: 9.38%
Train Epoch: 1 [320/1083 (29%)]	Loss: 0.700554	Accuracy: 9.90%
Train Epoch: 1 [352/1083 (32%)]	Loss: 0.698589	Accuracy: 8.85%
Train Epoch: 1 [384/1083 (35%)]	Loss: 0.700117	Accuracy: 8.85%
Train Epoch: 1 [416/1083 (38%)]	Loss: 0.698658	Accuracy: 9.38%
Train Epoch: 1 [448/1083 (41%)]	Loss: 0.701541	Accuracy: 10.42%
Train Epoc

# Model 2: DenseNet201

In [26]:
# DenseNet201

import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

In [27]:
from statistics import mean

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1:
                    denom += 1
                    if int(preds[i][j]) == int(target[i][j]):
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1:
                    denom += 1
                    if int(preds[n][m]) == int(target[n][m]):
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/1083 (0%)]	Loss: 0.702177	Accuracy: 9.90%
Train Epoch: 0 [32/1083 (3%)]	Loss: 0.678552	Accuracy: 10.94%
Train Epoch: 0 [64/1083 (6%)]	Loss: 0.660436	Accuracy: 7.29%
Train Epoch: 0 [96/1083 (9%)]	Loss: 0.637740	Accuracy: 2.60%
Train Epoch: 0 [128/1083 (12%)]	Loss: 0.620754	Accuracy: 1.04%
Train Epoch: 0 [160/1083 (15%)]	Loss: 0.599719	Accuracy: 1.04%
Train Epoch: 0 [192/1083 (18%)]	Loss: 0.575124	Accuracy: 2.60%
Train Epoch: 0 [224/1083 (21%)]	Loss: 0.556520	Accuracy: 0.00%
Train Epoch: 0 [256/1083 (24%)]	Loss: 0.538537	Accuracy: 1.56%
Train Epoch: 0 [288/1083 (26%)]	Loss: 0.529763	Accuracy: 1.56%
Train Epoch: 0 [320/1083 (29%)]	Loss: 0.506612	Accuracy: 0.00%
Train Epoch: 0 [352/1083 (32%)]	Loss: 0.496707	Accuracy: 0.00%
Train Epoch: 0 [384/1083 (35%)]	Loss: 0.468022	Accuracy: 0.00%
Train Epoch: 0 [416/1083 (38%)]	Loss: 0.462516	Accuracy: 0.00%
Train Epoch: 0 [448/1083 (41%)]	Loss: 0.442126	Accuracy: 1.04%
Train Epoch: 0 [480/1083 (44%)]	Loss: 0.439907	Accuracy: 0.00%


<ipython-input-27-d42b1c217955>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.2958 
Accuracy: 6.67%
Train Epoch: 1 [0/1083 (0%)]	Loss: 0.256802	Accuracy: 10.94%
Train Epoch: 1 [32/1083 (3%)]	Loss: 0.248416	Accuracy: 23.96%
Train Epoch: 1 [64/1083 (6%)]	Loss: 0.238551	Accuracy: 15.62%
Train Epoch: 1 [96/1083 (9%)]	Loss: 0.239712	Accuracy: 18.23%
Train Epoch: 1 [128/1083 (12%)]	Loss: 0.231030	Accuracy: 15.62%
Train Epoch: 1 [160/1083 (15%)]	Loss: 0.229046	Accuracy: 24.48%
Train Epoch: 1 [192/1083 (18%)]	Loss: 0.236136	Accuracy: 18.75%
Train Epoch: 1 [224/1083 (21%)]	Loss: 0.240231	Accuracy: 16.15%
Train Epoch: 1 [256/1083 (24%)]	Loss: 0.233078	Accuracy: 20.83%
Train Epoch: 1 [288/1083 (26%)]	Loss: 0.247776	Accuracy: 14.58%
Train Epoch: 1 [320/1083 (29%)]	Loss: 0.220907	Accuracy: 11.98%
Train Epoch: 1 [352/1083 (32%)]	Loss: 0.222410	Accuracy: 13.54%
Train Epoch: 1 [384/1083 (35%)]	Loss: 0.209298	Accuracy: 23.96%
Train Epoch: 1 [416/1083 (38%)]	Loss: 0.213027	Accuracy: 14.06%
Train Epoch: 1 [448/1083 (41%)]	Loss: 0.217702	Accuracy: 21.

# Model 3: ResNet152

In [22]:
# Resnet152 model
from torchvision import models

class ResNet152(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        #x = self.sigmoid(x)
        return x

In [23]:
model3 = ResNet152()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
from statistics import mean

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/1083 (0%)]	Loss: 0.926210	Accuracy: 41.67%
Train Epoch: 0 [32/1083 (3%)]	Loss: 0.911080	Accuracy: 40.10%
Train Epoch: 0 [64/1083 (6%)]	Loss: 0.901971	Accuracy: 16.15%
Train Epoch: 0 [96/1083 (9%)]	Loss: 0.885630	Accuracy: 24.48%
Train Epoch: 0 [128/1083 (12%)]	Loss: 0.876743	Accuracy: 6.77%
Train Epoch: 0 [160/1083 (15%)]	Loss: 0.863650	Accuracy: 11.46%
Train Epoch: 0 [192/1083 (18%)]	Loss: 0.848749	Accuracy: 3.12%
Train Epoch: 0 [224/1083 (21%)]	Loss: 0.835951	Accuracy: 2.08%
Train Epoch: 0 [256/1083 (24%)]	Loss: 0.827810	Accuracy: 6.25%
Train Epoch: 0 [288/1083 (26%)]	Loss: 0.816152	Accuracy: 3.12%
Train Epoch: 0 [320/1083 (29%)]	Loss: 0.807517	Accuracy: 3.12%
Train Epoch: 0 [352/1083 (32%)]	Loss: 0.799202	Accuracy: 3.12%
Train Epoch: 0 [384/1083 (35%)]	Loss: 0.791287	Accuracy: 0.00%
Train Epoch: 0 [416/1083 (38%)]	Loss: 0.778551	Accuracy: 3.65%
Train Epoch: 0 [448/1083 (41%)]	Loss: 0.773110	Accuracy: 1.04%
Train Epoch: 0 [480/1083 (44%)]	Loss: 0.765504	Accuracy: 1.

<ipython-input-24-3aabfdccf29b>:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.7059 
Accuracy: 0.00%
Train Epoch: 1 [0/1083 (0%)]	Loss: 0.704118	Accuracy: 0.00%
Train Epoch: 1 [32/1083 (3%)]	Loss: 0.703881	Accuracy: 1.04%
Train Epoch: 1 [64/1083 (6%)]	Loss: 0.703122	Accuracy: 0.00%
Train Epoch: 1 [96/1083 (9%)]	Loss: 0.701997	Accuracy: 0.00%
Train Epoch: 1 [128/1083 (12%)]	Loss: 0.701938	Accuracy: 4.17%
Train Epoch: 1 [160/1083 (15%)]	Loss: 0.701093	Accuracy: 0.00%
Train Epoch: 1 [192/1083 (18%)]	Loss: 0.700943	Accuracy: 4.17%
Train Epoch: 1 [224/1083 (21%)]	Loss: 0.699574	Accuracy: 9.38%
Train Epoch: 1 [256/1083 (24%)]	Loss: 0.698042	Accuracy: 7.29%
Train Epoch: 1 [288/1083 (26%)]	Loss: 0.697558	Accuracy: 5.73%
Train Epoch: 1 [320/1083 (29%)]	Loss: 0.700839	Accuracy: 1.04%
Train Epoch: 1 [352/1083 (32%)]	Loss: 0.697464	Accuracy: 10.42%
Train Epoch: 1 [384/1083 (35%)]	Loss: 0.695905	Accuracy: 8.85%
Train Epoch: 1 [416/1083 (38%)]	Loss: 0.696007	Accuracy: 12.50%
Train Epoch: 1 [448/1083 (41%)]	Loss: 0.696151	Accuracy: 12.50%
Train Ep